In [46]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

file_path = 'categorised_data.xlsx'
data = pd.read_excel(file_path)

In [48]:
category_row = data.iloc[1]

GenderSelect                      F
Country                           F
Age                               F
EmploymentStatus                  F
StudentStatus                     F
                                 ..
JobFactorRemote                   F
JobFactorIndustry                 F
JobFactorLeaderReputation         F
JobFactorDiversity                F
JobFactorPublishingOpportunity    F
Name: 1, Length: 91, dtype: object


In [49]:
category = category_row.to_dict()

In [51]:
fill_methods = data.iloc[2].to_dict()

In [53]:
data = data.iloc[3:].reset_index(drop=True)

,GenderSelect,Country,Age,EmploymentStatus,StudentStatus,LearningDataScience,CodeWriter,CareerSwitcher,CurrentJobTitleSelect,TitleFit,...,JobFactorExperienceLevel,JobFactorDepartment,JobFactorTitle,JobFactorCompanyFunding,JobFactorImpact,JobFactorRemote,JobFactorIndustry,JobFactorLeaderReputation,JobFactorDiversity,JobFactorPublishingOpportunity
0,Non-binary,NaN,NaN,Employed full-time,NaN,NaN,Yes,NaN,DBA/Database Engineer,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Female,United States,30,"Not employed, but looking for work",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somewhat important,NaN,NaN
2,Male,Cada,28,"Not employed, but looking for work",NaN,NaN,NaN,NaN,NaN,NaN,...,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important
3,Male,United States,56,"Independent contractor, freelancer, or self-em...",NaN,NaN,Yes,NaN,Operations Research Practitioner,Poorly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Male,Taiwan,38,Employed full-time,NaN,NaN,Yes,NaN,Computer Scientist,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# Fill blanks according to specified methods
for column, fill_method in fill_methods.items():
    if fill_method == 'Mode' or data[column].dtype == 'object':
        mode_value = data[column].mode()[0] if not data[column].mode().empty else 'Unknown'
        data[column] = data[column].fillna(mode_value)
    elif fill_method == 'Median':
        if data[column].dtype in ['float64', 'int64']:
            median_value = data[column].median()
            data[column] = data[column].fillna(median_value)
        else:
            mode_value = data[column].mode()[0] if not data[column].mode().empty else 'Unknown'
            data[column] = data[column].fillna(mode_value)
    else:
        data[column] = data[column].fillna(fill_method)

C:\Users\User\AppData\Local\Temp\ipykernel_21528\3319320218.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[column] = data[column].fillna(mode_value)


In [55]:
# Check if any blanks remain in the data
remaining_blanks = data.isnull().sum().sum()
if remaining_blanks > 0:
    print(f"There are {remaining_blanks} remaining blanks in the data.")
else:
    print("No blanks remain in the data.")

No blanks remain in the data.


In [56]:
# Identify numerical and non-numerical columns
numerical_columns = data.select_dtypes(include=[np.number]).columns
non_numerical_columns = data.select_dtypes(exclude=[np.number]).columns

In [57]:
# Process numerical columns
scaler = StandardScaler()
data[numerical_columns] = scaler.fit_transform(data[numerical_columns].astype(float))

In [58]:
# Rename numerical columns after scaling
renamed_numerical_columns = {column: f"{column}_{category.get(column, '')}" for column in numerical_columns}
data.rename(columns=renamed_numerical_columns, inplace=True)

In [59]:
# Initialize an empty DataFrame to store one-hot encoded data for non-numerical columns
one_hot_encoded_data = pd.DataFrame()

# Iterate over each non-numerical column to process them individually
for column in non_numerical_columns:
    # Split entries in the column by comma, expand to new DataFrame, and stack into a single Series
    expanded_data = data[column].str.split(',', expand=True).stack().reset_index(level=1, drop=True)
    # Convert categorical values in the Series to one-hot encoded format and sum the results for each original row index
    expanded_data = pd.get_dummies(expanded_data).groupby(level=0).sum()

    # Retrieve the category suffix for the current column from the category dictionary, default to empty string if not found
    category_suffix = category.get(column, '')
    # Rename the columns of the one-hot encoded DataFrame to include the category suffix
    expanded_data.columns = [f"{column}_{col}_{category_suffix}" for col in expanded_data.columns]
    
    # Concatenate the current one-hot encoded DataFrame to the accumulating DataFrame along columns
    one_hot_encoded_data = pd.concat([one_hot_encoded_data, expanded_data], axis=1)

In [61]:
# Concatenate numerical data and one-hot encoded data
final_numerical_columns = renamed_numerical_columns.values()
data = pd.concat([data[list(final_numerical_columns)], one_hot_encoded_data], axis=1)

In [62]:
processed_data_path = 'processed_data_final.csv'
data.to_csv(processed_data_path, index=False)

processed_data_path

'processed_data_final.csv'